# Small footprint proof of concept

The goal here is to test to what extent we can segment Ocean Parcels calculations.  In particular, can we run a long time span calculation trajectory calculation only having access to only a small subset of files at any given time.  This has two significant benefits:
1. smaller local storage is needed as input files are rotated in and out of the Fieldset and
2. smaller RAM is needed to store trajectory outputs since particle positions are written out incrementally.

Below, we run three cases:
1. Reference case (**ref**): OceanParcels has access to everything and runs all trajectories all at once,
2. Incremental data write only (**iwo**): OceanParcels has access to everything but runs incrementally and spits out trajectories incrementally, and
3. Incremental file read and incremental data write (**irw**): OceanParcels only can see the files it needs for each step and spits out trajectories incrementally.

## Dependencies

We start with importing the relevant modules

In [1]:
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4, ErrorCode
from datetime import timedelta
import numpy as np

INFO: Compiled ParcelsRandom ==> /tmp/parcels-40545/libparcels_random_d0a4842e-e318-46ba-a5a7-feb6c8600376.so


## Input velocity fields for reference case

We will use the Globcurrent fields from the `GlobCurrent_example_data` directory leveraging the fact that all filenames are nearly the same format with `<YYYY><MM><DD><HH><MM><SS>-GLOBCURRENT...`.  In this case, it is pretty simple because all files represent 1 day at the same time each day (uniform time increment).  Right here, note that OceanParcels has access to all files for this baseline case.

In [2]:
filenames = "/pw/workflows/ocean_parcels_demo/parcels_examples/GlobCurrent_example_data/20*.nc"
variables = {'U': 'eastward_eulerian_current_velocity',
             'V': 'northward_eulerian_current_velocity'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}
fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

## Initialize particle set

Now create vectors of Longitude and Latitude starting locations on a regular mesh, and use these to initialise a `ParticleSet` object.

In [3]:
lons, lats = np.meshgrid(range(15, 35), range(-40, -30))
pset = ParticleSet(fieldset=fieldset, pclass=JITParticle, lon=lons, lat=lats)

## Output file for reference case

Specify the file name and output time step

In [4]:
ref_out_file = pset.ParticleFile(name="ref.nc", outputdt=timedelta(hours=6))

Now we want to advect the particles. However, the Globcurrent data that we loaded in is only for a limited, regional domain and particles might be able to leave this domain. We therefore need to tell Parcels that particles that leave the domain need to be deleted. We do that using a `Recovery Kernel`, which will be invoked when a particle encounters an `ErrorOutOfBounds` error:

In [5]:
def DeleteParticle(particle, fieldset, time):
    particle.delete()
    
def StopParticle(particle, fieldset, time):
    particle.stop()

## Advect particles for reference case

Now we can advect the particles. Note that we do this inside a `for`-loop, so we can save a plot every six hours (which is the value of `runtime`).

In [6]:
# Inputs: pset, DeleteParticle
# Outputs: list of files (each file name is in savefile)
for cnt in range(20):
    # Set filename for output plot
    output_image = 'particles'+str(cnt).zfill(2)
    print(output_image)
    
    # First plot the particles
    pset.show(savefile=output_image, field='vector', land=True, vmax=2.0)

    # Then advect the particles for 6 hours
    pset.execute(AdvectionRK4,
                 runtime=timedelta(hours=6),  # runtime controls the interval of the plots
                 dt=timedelta(minutes=5),
                 output_file=ref_out_file,
                 recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})  # the recovery kernel

In /pw/.miniconda3/envs/parsl-pw/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /pw/.miniconda3/envs/parsl-pw/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /pw/.miniconda3/envs/parsl-pw/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /pw/.miniconda3/envs/parsl-pw/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /pw/.miniconda3/envs/parsl-pw

particles00


/pw/.miniconda3/envs/parsl-pw/lib/python3.7/site-packages/parcels/plotting.py:191: RuntimeWarning: invalid value encountered in true_divide
  u = np.where(speed > 0., data[0]/speed, 0)
/pw/.miniconda3/envs/parsl-pw/lib/python3.7/site-packages/parcels/plotting.py:192: RuntimeWarning: invalid value encountered in true_divide
  v = np.where(speed > 0., data[1]/speed, 0)
/pw/.miniconda3/envs/parsl-pw/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:388: MatplotlibDeprecationWarning: 
The 'inframe' parameter of draw() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use Axes.redraw_in_frame() instead. If any parameter follows 'inframe', they should be passed as keyword, not positionally.
  inframe=inframe)
INFO: Plot saved to particles00.png
INFO: Compiled JITParticleAdvectionRK4 ==> /tmp/parcels-40545/db3e0435be9a3000d7c60dda7171aef1_0.so


particles01


INFO: Plot saved to particles01.png


...

INFO: Plot saved to particles17.png


particles18


INFO: Plot saved to particles18.png


particles19


INFO: Plot saved to particles19.png


## Write output file for reference case

In [7]:
ref_out_file.export()